In [1]:
from tqdm import tqdm
from shapely.wkt import dumps
import orjson

In [2]:
%%time
with open('13_266069_040_003 L02 PAS.json', 'r') as file:
    data = orjson.loads(file.read())

import shapely
from shapely.geometry import shape

CPU times: user 8.05 s, sys: 1.1 s, total: 9.15 s
Wall time: 9.91 s


In [3]:
len(data),data[0]

(88605,
 {'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[45837, 20092],
     [45836, 20093],
     [45835, 20094],
     [45834, 20095],
     [45833, 20096],
     [45832, 20096],
     [45831, 20096],
     [45830, 20096],
     [45829, 20096],
     [45828, 20096],
     [45827, 20096],
     [45826, 20096],
     [45825, 20096],
     [45824, 20096],
     [45823, 20096],
     [45822, 20096],
     [45821, 20096],
     [45820, 20097],
     [45819, 20098],
     [45818, 20099],
     [45817, 20100],
     [45816, 20100],
     [45815, 20100],
     [45814, 20100],
     [45813, 20100],
     [45812, 20101],
     [45811, 20102],
     [45810, 20103],
     [45809, 20104],
     [45808, 20104],
     [45807, 20104],
     [45806, 20104],
     [45805, 20104],
     [45804, 20105],
     [45803, 20106],
     [45802, 20107],
     [45801, 20108],
     [45800, 20109],
     [45799, 20110],
     [45798, 20111],
     [45797, 20112],
     [45796, 20113],
     [45795, 20114],
     [45794, 20115]

### Convert Geojson to String (orjson)

In [4]:
%%time
for _ in range(12):
        for geojson in tqdm(data):
            name = geojson["properties"]["classification"]["name"]
            geometry = orjson.dumps(geojson["geometry"]).decode('ascii')

100%|██████████████████████████████████████████████████████████████████████████████████████| 88605/88605 [00:00<00:00, 219463.82it/s]

CPU times: user 5.16 s, sys: 33.9 ms, total: 5.19 s
Wall time: 5.19 s


### Convert geojson to shapely

In [5]:
%%time
for _ in range(12):
    for geojson in tqdm(data):
        name = geojson["properties"]["classification"]["name"]
        shapely_geom = shape(geojson["geometry"])

100%|███████████████████████████████████████████████████████████████████████████████████████| 88605/88605 [00:08<00:00, 10670.41it/s]

CPU times: user 1min 42s, sys: 1.9 s, total: 1min 44s
Wall time: 1min 43s


### Convert Geojson to Shapely to WKT

In [6]:
%%time
for _ in range(12):
    for geojson in tqdm(data):
        name = geojson["properties"]["classification"]["name"]
        shapely_geom = shape(geojson["geometry"])
        wkt=shapely_geom.wkt

100%|████████████████████████████████████████████████████████████████████████████████████████| 88605/88605 [00:11<00:00, 7904.98it/s]

CPU times: user 2min 15s, sys: 2.73 s, total: 2min 18s
Wall time: 2min 17s


### Convert Geojson to WKB

In [7]:
%%time
for _ in range(12):
    for geojson in tqdm(data):
        name = geojson["properties"]["classification"]["name"]
        shapely_geom = shape(geojson["geometry"])
        wkb=shapely_geom.wkb

100%|████████████████████████████████████████████████████████████████████████████████████████| 88605/88605 [00:10<00:00, 8817.43it/s]

CPU times: user 2min, sys: 2.5 s, total: 2min 2s
Wall time: 2min 1s


## Convert WKB to Shapely object

### first generate WKB from geojsons

In [8]:
%%time
wkbs=[]
for geojson in tqdm(data):
    name = geojson["properties"]["classification"]["name"]
    shapely_geom = shape(geojson["geometry"])
    wkbs.append(shapely_geom.wkb)

100%|████████████████████████████████████████████████████████████████████████████████████████| 88605/88605 [00:11<00:00, 7902.32it/s]

CPU times: user 11.1 s, sys: 160 ms, total: 11.2 s
Wall time: 11.2 s


### now convert the WKB back to shapely objects

In [9]:
%%time
import shapely.wkb
for _ in range(12):
    for wkb in tqdm(wkbs):
        shapely_geom = shapely.wkb.loads(wkb)

100%|██████████████████████████████████████████████████████████████████████████████████████| 88605/88605 [00:00<00:00, 177166.32it/s]

CPU times: user 5.95 s, sys: 39.8 ms, total: 5.99 s
Wall time: 5.98 s


# Check multiproc

In [10]:
import ray

In [11]:
ray.init()

2024-10-31 16:07:29,765	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.37.0
Dashboard:,http://127.0.0.1:8265


In [12]:
ray.cluster_resources()

{'accelerator_type:G': 1.0,
 'node:__internal_head__': 1.0,
 'CPU': 16.0,
 'object_store_memory': 8101124505.0,
 'memory': 16202249012.0,
 'node:172.17.185.211': 1.0,
 'GPU': 1.0}

In [13]:
@ray.remote
def bulk_convert(geojsongeoms):
    wkts=[]
    for geojsongeom in geojsongeoms:
        shapely_geom = shape(geojsongeom)
        wkt=shapely_geom.wkt
        wkts.append(wkt)
    return(wkts)

In [14]:
%%time
futures = [] 
for _ in range(12):
    batch_size=10_000
    polygons=[]

    for geojson in tqdm(data):
        name = geojson["properties"]["classification"]["name"]
        geom = geojson["geometry"]
    
        polygons.append(geom)
    
        if len(polygons) == batch_size:
            futures.append(bulk_convert.remote(polygons))
            polygons=[]
            
    if polygons:
        futures.append(bulk_convert.remote(polygons))
    
for f in tqdm(futures):
    ray.get(f)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 34.51it/s]

CPU times: user 37.4 s, sys: 1.49 s, total: 38.9 s
Wall time: 40.1 s
